In [22]:
from calcula_resultados import compute_indicators_from_files, load_balancetes_to_df, extract_accounts
import pandas as pd
import json
from utils.functions import processar_indicadores_financeiros, extract_accounts
files = ["balancetes/balancete1.json"]
result = compute_indicators_from_files(files)

In [11]:
# ---- DataFrame por arquivo ----
df_balancete = pd.json_normalize(
    result["per_file"]).drop(columns=["file", "n_leaves"])


df_balancete

,indicators.receita_bruta,indicators.impostos_vendas,indicators.receita_liquida,indicators.custos,indicators.lucro_bruto,indicators.despesas,indicators.lucro_liquido,indicators.disponibilidade_caixa
0,684563.19,3160.22,681402.97,405696.63,275706.34,416239.67,-140533.33,130876.58


In [ ]:
# ==== LEITURA DE UM JSON ====
with open("balancetes/nfeat/Balancete.2023-01.normalized.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [2]:
# ==== FUNÇÃO TRANSFORMADORA ====
def extract_accounts(node, hierarchy=None):
    if hierarchy is None:
        hierarchy = []

    current_hierarchy = hierarchy + [node["descricao"]]
    rows = []

    if "children" in node:
        for child in node["children"]:
            rows.extend(extract_accounts(child, current_hierarchy))
    else:
        row = {f"nivel_{i+1}": level for i,
               level in enumerate(current_hierarchy)}
        row["conta"] = node["conta"]
        row["descricao"] = node["descricao"]
        row["saldo_atual"] = node.get("saldo_atual", 0.0)
        rows.append(row)

    return rows

In [4]:
all_rows = []
for section in data.values():
    if isinstance(section, dict) and "descricao" in section:
        all_rows.extend(extract_accounts(section))

# Create a DataFrame
df = pd.DataFrame(all_rows)

In [21]:
# ==== LÊ TODOS OS BALANCETES ====
import os
import json
import pandas as pd

# Caminho da pasta
folder_path = "balancetes/industrial_nordeste"

all_rows = []

# Loop em todos os arquivos JSON da pasta
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)

        # Extrai o mês do nome do arquivo (ex: Balancete.2023-01.normalized.json → 2023-01)
        mes = filename.split(".")[1]

        # Lê o JSON
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        # Percorre as seções e extrai as contas
        for section in data.values():
            if isinstance(section, dict) and "descricao" in section:
                contas = extract_accounts(section)
                # Adiciona o mês a cada conta extraída
                for conta in contas:
                    conta["mes"] = mes
                all_rows.extend(contas)

# Cria o DataFrame consolidado
df = pd.DataFrame(all_rows)



In [23]:
tabela_pivot = processar_indicadores_financeiros(df)

In [ ]:

df[df["nivel_1"].str.startswith("A")]

,nivel_1,nivel_2,nivel_3,nivel_4,conta,descricao,saldo_atual


In [59]:
df.drop_duplicates(subset=['nivel_2', 'nivel_3'])

,nivel_1,nivel_2,nivel_3,nivel_4,nivel_5,conta,descricao,saldo_atual,mes
0,ATIVO,ATIVO CIRCULANTE,DISPONIBILIDADES,CAIXA,Caixa,01.1.1.01.001,Caixa,109463.73,2023-01
1,ATIVO,ATIVO CIRCULANTE,DIREITOS REALIZÁVEIS A CURTO PRAZO,CLIENTES NACIONAIS,Clientes nacionais,01.1.2.01.001,Clientes nacionais,34968.86,2023-01
5,ATIVO,ATIVO NÃO CIRCULANTE,IMOBILIZADO,BENS E DIREITOS EM USO,Máquinas e Equipamentos,01.2.3.01.001,Máquinas e Equipamentos,20355.33,2023-01
7,PASSIVO,PASSIVO CIRCULANTE,FORNECEDORES,REZENDE CENTRO COMERCIAL AUTOMOTIVO LTDA,NaN,02.1.1.01.000007895,REZENDE CENTRO COMERCIAL AUTOMOTIVO LTDA,2106.89,2023-01
12,PASSIVO,PASSIVO CIRCULANTE,EMPRÉSTIMOS E FINANCIAMENTOS,Banco Bradesco S/A,NaN,02.1.2.01.010,Banco Bradesco S/A,23552.35,2023-01
13,PASSIVO,PASSIVO CIRCULANTE,OBRIGAÇÕES TRABALHISTAS,Salários a pagar,NaN,02.1.3.01.001,Salários a pagar,43593.85,2023-01
15,PASSIVO,PASSIVO CIRCULANTE,OBRIGAÇÕES TRIBUTÁRIAS,IRRF a Recolher,NaN,02.1.4.01.001,IRRF a Recolher,557.99,2023-01
20,PASSIVO,PASSIVO NÃO CIRCULANTE,Bens Remetidos para Teste,NaN,NaN,02.2.1.02.002,Bens Remetidos para Teste,14602.65,2023-01
21,PASSIVO,PATRIMÔNIO LÍQUIDO,Capital Social Integralizado,NaN,NaN,02.3.1.01.002,Capital Social Integralizado,8740.59,2023-01
22,PASSIVO,PATRIMÔNIO LÍQUIDO,Reserva de Lucros,NaN,NaN,02.3.2.02.001,Reserva de Lucros,71143.86,2023-01


In [19]:
tabela_pivot

,ATIVO CIRCULANTE,ATIVO NÃO CIRCULANTE,PASSIVO CIRCULANTE,PASSIVO NÃO CIRCULANTE,Receita Bruta,(-) Impostos sobre Receita,Custo Total,Disponibilidade de Caixa,Receita Líquida,Lucro Bruto,Lucro Líquido,Ativo_Total,Passivo_Total,Liquidez_Corrente,Liquidez_Imediata,Liquidez_Geral,Solvencia_Geral,Endividamento,Endividamento_Geral
mes,,,,,,,,,,,,,,,,,,,
2022-01,150278.15,19171.41,111293.28,12521.64,348677.37,-34928.03,312009.88,83957.77,383605.40,71595.52,71595.52,169449.56,123814.92,1.350290,0.754383,1.368571,1.368571,0.730689,0.730689
2022-02,141171.77,18009.69,104549.25,11762.87,327548.66,-32811.51,293103.11,78870.20,360360.17,67257.06,67257.06,159181.46,116312.12,1.350290,0.754383,1.368572,1.368572,0.730689,0.730689
2022-03,144877.35,18482.41,107293.53,12071.63,336146.37,-33672.76,300796.66,80940.44,369819.13,69022.47,69022.47,163359.76,119365.16,1.350290,0.754383,1.368572,1.368572,0.730689,0.730689
2022-04,144109.33,18384.44,106724.75,12007.63,334364.42,-33494.26,299202.11,80511.36,367858.68,68656.57,68656.57,162493.77,118732.38,1.350290,0.754383,1.368572,1.368572,0.730689,0.730689
2022-05,151716.46,19354.90,112358.44,12641.48,352014.56,-35262.33,314996.13,84761.33,387276.89,72280.76,72280.76,171071.36,124999.92,1.350290,0.754383,1.368572,1.368572,0.730689,0.730689
2022-06,173455.15,22128.16,128457.72,14452.82,402452.98,-40314.89,360130.36,96906.36,442767.87,82637.51,82637.51,195583.31,142910.54,1.350290,0.754383,1.368572,1.368572,0.730689,0.730689
2022-07,154024.22,19649.31,114067.55,12833.77,357369.10,-35798.71,319787.58,86050.64,393167.81,73380.23,73380.23,173673.53,126901.32,1.350290,0.754383,1.368572,1.368572,0.730689,0.730689
2022-08,142089.63,18126.78,105229.00,11839.35,329678.26,-33024.84,295008.75,79382.99,362703.10,67694.35,67694.35,160216.41,117068.35,1.350290,0.754383,1.368572,1.368572,0.730689,0.730689
2022-09,147054.48,18760.16,108905.87,12253.03,341197.75,-34178.78,305316.83,82156.76,375376.53,70059.70,70059.70,165814.64,121158.90,1.350290,0.754383,1.368572,1.368572,0.730689,0.730689


In [20]:
ultima_data = tabela_pivot.index.max()
indices_max = tabela_pivot.loc[tabela_pivot.index == ultima_data]
indices_max

,ATIVO CIRCULANTE,ATIVO NÃO CIRCULANTE,PASSIVO CIRCULANTE,PASSIVO NÃO CIRCULANTE,Receita Bruta,(-) Impostos sobre Receita,Custo Total,Disponibilidade de Caixa,Receita Líquida,Lucro Bruto,Lucro Líquido,Ativo_Total,Passivo_Total,Liquidez_Corrente,Liquidez_Imediata,Liquidez_Geral,Solvencia_Geral,Endividamento,Endividamento_Geral
mes,,,,,,,,,,,,,,,,,,,
2024-12,241878.25,30857.09,179130.61,20154.04,561209.16,-56217.96,502191.49,135133.14,617427.12,115235.63,115235.63,272735.34,199284.65,1.35029,0.754383,1.368572,1.368572,0.730689,0.730689


In [47]:
df_agrupado.head(50)

,nivel_2,mes,saldo_atual
0,ATIVO CIRCULANTE,2023-01,171846.85
1,ATIVO CIRCULANTE,2023-02,194311.17
2,ATIVO CIRCULANTE,2023-03,195146.77
3,ATIVO CIRCULANTE,2023-04,180656.01
4,ATIVO CIRCULANTE,2023-05,169029.08
5,ATIVO CIRCULANTE,2023-06,176836.56
6,ATIVO CIRCULANTE,2023-07,177204.61
7,ATIVO CIRCULANTE,2023-08,158219.08
8,ATIVO CIRCULANTE,2023-09,186976.07
9,ATIVO CIRCULANTE,2023-10,178534.83


In [ ]:
import json
import random


def modificar_valores_json(json_data):
    def modificar_nodo(nodo):
        # Se o nó tiver filhos, processa-os recursivamente
        if 'children' in nodo:
            for filho in nodo['children']:
                modificar_nodo(filho)
            return

        # Modifica os valores financeiros com variação de ±25%
        for campo in ['saldo_anterior', 'debito', 'credito']:
            if campo in nodo:
                valor_original = nodo[campo]
                # Gera uma variação aleatória entre -25% e +25%
                variacao = random.uniform(-0.25, 0.25)
                novo_valor = valor_original * (1 + variacao)
                nodo[campo] = round(novo_valor, 2)

        # Calcula o novo saldo_atual baseado no tipo de conta
        if 'conta' in nodo:
            conta = nodo['conta']
            if conta.startswith(('01', '04')):  # ATIVO e CUSTOS
                nodo['saldo_atual'] = round(nodo.get('saldo_anterior', 0) +
                                            nodo.get('debito', 0) -
                                            nodo.get('credito', 0), 2)
            elif conta.startswith(('02', '03')):  # PASSIVO e RECEITAS
                nodo['saldo_atual'] = round(nodo.get('saldo_anterior', 0) -
                                            nodo.get('debito', 0) +
                                            nodo.get('credito', 0), 2)

    # Processa cada seção do JSON
    for secao in json_data.values():
        if isinstance(secao, dict) and 'conta' in secao:
            modificar_nodo(secao)

    return json_data

In [35]:
# ==== LEITURA DE UM JSON ====
with open("balancetes/nfeat/Balancete.2023-12.normalized.json", "r", encoding="utf-8") as f:
    json_original = json.load(f)

json_modificado = modificar_valores_json(json_original)

# Salva o arquivo
with open('balancetes/industrial_nordeste/Balancete.2023-12.json', 'w', encoding='utf-8') as arquivo:
    json.dump(json_modificado, arquivo, indent=4)